In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/home-credit-default-risk/application_train.csv')
test=pd.read_csv('../input/home-credit-default-risk/application_test.csv')

train set 관련 정보 출력

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
train.info(verbose=True, null_counts=True)

In [ ]:
test.isna().sum()

In [ ]:
target=train['TARGET']
test_id = test[['SK_ID_CURR']].copy()

범주형 컬럼 -> 라벨 인코딩   
simple imputer -> 결측치(nan) 대치

In [ ]:
train=train.drop(columns=['SK_ID_CURR', 'TARGET'])
test=test.drop(columns='SK_ID_CURR')
cols=train.columns
test_cols=test.columns

from sklearn.preprocessing import LabelEncoder


for i in train.columns:
    le = LabelEncoder()
    train[i]=le.fit_transform(train[i])

for i in test.columns:
    le = LabelEncoder()
    test[i]=le.fit_transform(test[i])

In [ ]:
from sklearn.impute import SimpleImputer

transformer = SimpleImputer(strategy='median')
train=transformer.fit_transform(train)

transformer = SimpleImputer(strategy='median')
test=transformer.fit_transform(test)

In [ ]:
train_data = pd.DataFrame(train, columns=cols)
test_data = pd.DataFrame(test, columns=test_cols)

In [ ]:
train_data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

for col in train_data.columns:
    train_data[[col]] = scaler.fit_transform(train_data[[col]])
    

for col in test_data.columns:
    test_data[[col]] = scaler.fit_transform(test_data[[col]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_estimators=250, min_samples_split=16, min_samples_leaf=18, random_state=0, n_jobs=-1)
rf.fit(train_data,target)


In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
    nthread=4,
    n_estimators=10000,
    learning_rate=0.02,
    num_leaves=34,
    colsample_bytree=0.9497036,
    subsample=0.8715623,
    max_depth=8,
    reg_alpha=0.041545473,
    reg_lambda=0.0735294,
    min_split_gain=0.0222415,
    min_child_weight=39.3259775,
    silent=-1,
    verbose=-1, )

clf.fit(train_data, target, eval_set=[(train_data, target)],
        eval_metric='auc', verbose=200)

In [ ]:
arr=clf.predict_proba(test_data)
arr.shape

In [ ]:
arr[:,1].shape

lgbm acc: 0.71606
-> 파라미터 조정 필요

In [ ]:
result_lgbm_model = pd.DataFrame(clf.predict_proba(test_data)[:, 1], columns=['TARGET'])
result_lgbm = pd.concat([test_id, result_lgbm_model], axis=1)
RESULT_lgbm = result_lgbm.set_index('SK_ID_CURR')
RESULT_lgbm.to_csv('result_lgbm.csv')

rf acc:0.72398

In [ ]:
result_rf_model = pd.DataFrame(rf.predict_proba(test_data)[:, 1], columns=['TARGET'])
result_rf = pd.concat([test_id, result_rf_model], axis=1)
RESULT_rf = result_rf.set_index('SK_ID_CURR')
RESULT_rf.to_csv('result rf.csv')

In [ ]:
print('done')